In [1]:
%pip install langgraph-sdk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from pprint import pprint

In [4]:
from langgraph_sdk import get_sync_client

client = get_sync_client(url="http://127.0.0.1:2024")

## Client for managing assistants in LangGraph
---

This class provides methods to interact with assistants, which are versioned configurations of your graph.

Example:
```
client = get_client()
assistant = await client.assistants.get("assistant_id_123")
```

### Create a new assistant

Useful when graph is configurable and you want to create different assistants based on different configurations.

**Parameters:**

- **`graph_id`** (`Optional[str]`) – The ID of the graph the assistant should use. The graph ID is normally set in your langgraph.json configuration.
- **`config`** (`Optional[Config]`, default: `None` ) – Configuration to use for the graph.
- **`metadata`** (`Json`, default: None` ) – Metadata to add to assistant.
- **`assistant_id`** (`Optional[str]`, default: `None` ) – Assistant ID to use, will default to a random UUID if not provided.
- **`if_exists`** (`Optional[OnConflictBehavior]`, default: `None` ) – How to handle duplicate creation. Defaults to 'raise' under the hood. Must be either 'raise' (raise error if duplicate), or 'do_nothing' (return existing assistant).
- **`name`** (`Optional[str]`, default: `None` ) – The name of the assistant. Defaults to 'Untitled' under the hood.
- **`headers`** (`Optional[dict[str, str]]`, default: `None` ) – Optional custom headers to include with the request.

**Returns:**

- **`Assistant`** ( `Assistant` ) – The created assistant.

Example Usage:

```
assistant = await client.assistants.create(
    graph_id="agent",
    config={"configurable": {"model_name": "openai"}},
    metadata={"number":1},
    assistant_id="my-assistant-id",
    if_exists="do_nothing",
    name="my_name"
)
```

In [21]:
assistant = client.assistants.create(graph_id="retrieval_graph")
pprint(assistant)

{'assistant_id': '631035d1-7174-4fed-a12b-150fc4e8f4e6',
 'config': {},
 'created_at': '2025-04-01T14:51:10.627665+00:00',
 'graph_id': 'retrieval_graph',
 'metadata': {},
 'name': 'Untitled',
 'updated_at': '2025-04-01T14:51:10.627665+00:00',
 'version': 1}


### Update an assistant

Use this to point to a different graph, update the configuration, or change the metadata of an assistant.

**Parameters:**

- **`assistant_id`** (`str`) – Assistant to update.
- **`graph_id`** (`Optional[str]`, default: `None` ) – The ID of the graph the assistant should use. The graph ID is normally set in your langgraph.json configuration. If None, assistant will keep pointing to same graph.
- **`config`** (`Optional[Config]`, default: `None` ) – Configuration to use for the graph.
- **`metadata`** (`Json`, default: `None` ) – Metadata to merge with existing assistant metadata.
- **`name`** (`Optional[str]`, default: `None` ) – The new name for the assistant.
- **`headers`** (`Optional[dict[str, str]]`, default: `None` ) – Optional custom headers to include with the request.

**Returns:**

**`Assistant`** ( `Assistant` ) – The updated assistant.

Example Usage:

```
assistant = await client.assistants.update(
    assistant_id='e280dad7-8618-443f-87f1-8e41841c180f',
    graph_id="other-graph",
    config={"configurable": {"model_name": "anthropic"}},
    metadata={"number":2}
)
```

In [22]:
assistant = client.assistants.update(
    assistant_id=assistant["assistant_id"],
    config={
        "configurable": {
            "query_model": "openai/gpt-4o",
            "response_model": "openai/gpt-4o",
            "retriever_provider": "mongodb",
        }
    },
)
pprint(assistant)

{'assistant_id': '631035d1-7174-4fed-a12b-150fc4e8f4e6',
 'config': {'configurable': {'query_model': 'openai/gpt-4o',
                             'response_model': 'openai/gpt-4o',
                             'retriever_provider': 'mongodb'}},
 'created_at': '2025-04-01T14:51:10.627665+00:00',
 'graph_id': 'retrieval_graph',
 'metadata': {},
 'name': 'Untitled',
 'updated_at': '2025-04-01T14:58:55.549476+00:00',
 'version': 2}


### Get the schemas of an assistant by ID

**Parameters:**

- **`assistant_id`** (`str`) – The ID of the assistant to get the schema of.
- **`headers`** (`Optional[dict[str, str]]`, default: `None` ) – Optional custom headers to include with the request.

**Returns:**

- **`GraphSchema`** ( `GraphSchema` ) – The graph schema for the assistant.

Example Usage:
```
schema = await client.assistants.get_schemas(
    assistant_id="my_assistant_id"
)
print(schema)

----------------------------------------------------------------------------------------------------------------------------

{
    'graph_id': 'agent',
    'state_schema':
        {
            'title': 'LangGraphInput',
            '$ref': '#/definitions/AgentState',
            'definitions':
                {
                    'BaseMessage':
                        {
                            'title': 'BaseMessage',
                            'description': 'Base abstract Message class. Messages are the inputs and outputs of ChatModels.',
                            'type': 'object',
                            'properties':
                                {
                                 'content':
                                    {
                                        'title': 'Content',
                                        'anyOf': [
                                            {'type': 'string'},
                                            {'type': 'array','items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}
                                        ]
                                    },
                                'additional_kwargs':
                                    {
                                        'title': 'Additional Kwargs',
                                        'type': 'object'
                                    },
                                'response_metadata':
                                    {
                                        'title': 'Response Metadata',
                                        'type': 'object'
                                    },
                                'type':
                                    {
                                        'title': 'Type',
                                        'type': 'string'
                                    },
                                'name':
                                    {
                                        'title': 'Name',
                                        'type': 'string'
                                    },
                                'id':
                                    {
                                        'title': 'Id',
                                        'type': 'string'
                                    }
                                },
                            'required': ['content', 'type']
                        },
                    'AgentState':
                        {
                            'title': 'AgentState',
                            'type': 'object',
                            'properties':
                                {
                                    'messages':
                                        {
                                            'title': 'Messages',
                                            'type': 'array',
                                            'items': {'$ref': '#/definitions/BaseMessage'}
                                        }
                                },
                            'required': ['messages']
                        }
                }
        },
    'config_schema':
        {
            'title': 'Configurable',
            'type': 'object',
            'properties':
                {
                    'model_name':
                        {
                            'title': 'Model Name',
                            'enum': ['anthropic', 'openai'],
                            'type': 'string'
                        }
                }
        }
}
```

In [14]:
schema = client.assistants.get_schemas(assistant_id=assistant["assistant_id"])
pprint(schema)

{'config_schema': {'properties': {'embedding_model': {'default': 'openai/text-embedding-3-small',
                                                      'description': 'Name of '
                                                                     'the '
                                                                     'embedding '
                                                                     'model to '
                                                                     'use. '
                                                                     'Must be '
                                                                     'a valid '
                                                                     'embedding '
                                                                     'model '
                                                                     'name.',
                                                      'title': 'Embedding '
                                

### Delete an assistant

**Parameters:**

- **`assistant_id`** (`str`) – The assistant ID to delete.
- **`headers`** (`Optional[dict[str, str]]`, default: `None` ) – Optional custom headers to include with the request.

**Returns:**

- `None` – None

Example Usage:
```
await client.assistants.delete(
    assistant_id="my_assistant_id"
)
```

In [16]:
client.assistants.delete(assistant_id=assistant["assistant_id"])

### Search for assistants

**Parameters:**

- **`metadata`** (`Json`, default: `None` ) – Metadata to filter by. Exact match filter for each KV pair.
- **`graph_id`** (`Optional[str]`, default: `None` ) – The ID of the graph to filter by. The graph ID is normally set in your langgraph.json configuration.
- **`limit`** (`int`, default: `10` ) – The maximum number of results to return.
- **`offset`** (`int`, default: `0` ) – The number of results to skip.
- **`headers`** (`Optional[dict[str, str]]`, default: `None` ) – Optional custom headers to include with the request.

**Returns:**

- `list[Assistant]` – list[Assistant]: A list of assistants.

Example Usage:
```
assistants = await client.assistants.search(
    metadata = {"name":"my_name"},
    graph_id="my_graph_id",
    limit=5,
    offset=5
)
```

In [17]:
result = client.assistants.search(graph_id="retrieval_graph")
pprint(result)

[{'assistant_id': '571ade52-f5cd-582a-89d9-d79dc861a8ba',
  'config': {},
  'created_at': '2025-04-01T04:44:02.720547+00:00',
  'graph_id': 'retrieval_graph',
  'metadata': {'created_by': 'system'},
  'name': 'retrieval_graph',
  'updated_at': '2025-04-01T04:44:02.720547+00:00',
  'version': 1}]


## Client for managing threads in LangGraph.
---

A thread maintains the state of a graph across multiple interactions/invocations (aka runs). It accumulates and persists the graph's state, allowing for continuity between separate invocations of the graph.

Example:
```
client = get_client()
new_thread = await client.threads.create(metadata={"user_id": "123"})
```

### Create a new thread

**Parameters:**

- **`metadata`** (`Json`, default: `None` ) – Metadata to add to thread.
- **`thread_id`** (`Optional[str]`, default: `None` ) – ID of thread. If None, ID will be a randomly generated UUID.
- **`if_exists`** (`Optional[OnConflictBehavior]`, default: `None` ) – How to handle duplicate creation. Defaults to 'raise' under the hood. Must be either 'raise' (raise error if duplicate), or 'do_nothing' (return existing thread).
- **`supersteps`** (`Optional[Sequence[dict[str, Sequence[dict[str, Any]]]]]`, default: `None` ) – Apply a list of supersteps when creating a thread, each containing a sequence of updates. Each update has values or command and as_node. Used for copying a thread between deployments.
- **`graph_id`** (`Optional[str]`, default: `None` ) – Optional graph ID to associate with the thread.
- **`headers`** (`Optional[dict[str, str]]`, default: `None` ) – Optional custom headers to include with the request.

**Returns:**

- **`Thread`** ( `Thread` ) – The created thread.

Example Usage:
```
thread = await client.threads.create(
    metadata={"number":1},
    thread_id="my-thread-id",
    if_exists="raise"
)
```

In [19]:
thread = client.threads.create()
pprint(thread)

{'config': {},
 'created_at': '2025-04-01T14:41:06.642018+00:00',
 'metadata': {},
 'status': 'idle',
 'thread_id': 'b7765357-9b53-44a9-87cc-8e427f02cce5',
 'updated_at': '2025-04-01T14:41:06.642022+00:00',
 'values': None}


### Get a thread by ID.

**Parameters:**

- **`thread_id`** (`str`) – The ID of the thread to get.
- **`headers`** (`Optional[dict[str, str]]`, default: `None` ) – Optional custom headers to include with the request.

**Returns:**

- **`Thread`** ( `Thread` ) – Thread object.

Example Usage:
```
thread = await client.threads.get(
    thread_id="my_thread_id"
)
print(thread)

-----------------------------------------------------

{
    'thread_id': 'my_thread_id',
    'created_at': '2024-07-18T18:35:15.540834+00:00',
    'updated_at': '2024-07-18T18:35:15.540834+00:00',
    'metadata': {'graph_id': 'agent'}
}
```

In [20]:
thread = client.threads.get(thread_id=thread["thread_id"])
pprint(thread)

{'config': {},
 'created_at': '2025-04-01T14:41:06.642018+00:00',
 'metadata': {},
 'status': 'idle',
 'thread_id': 'b7765357-9b53-44a9-87cc-8e427f02cce5',
 'updated_at': '2025-04-01T14:41:06.642022+00:00',
 'values': None}


## Client for managing runs in LangGraph
---

A run is a single assistant invocation with optional input, config, and metadata. This client manages runs, which can be stateful (on threads) or stateless.

Example:
```
client = get_client()
run = await client.runs.create(assistant_id="asst_123", thread_id="thread_456", input={"query": "Hello"})
```

### Create a run and stream the results

**Parameters:**

- **`thread_id`** (`Optional[str]`) – the thread ID to assign to the thread. If None will create a stateless run.
- **`assistant_id`** (`str`) – The assistant ID or graph name to stream from. If using graph name, will default to first assistant created from that graph.
- **`input`** (`Optional[dict]`, default: `None` ) – The input to the graph.
- **`command`** (`Optional[Command]`, default: `None` ) – A command to execute. Cannot be combined with input.
- **`stream_mode`** (`Union[StreamMode, Sequence[StreamMode]]`, default: `'values'` ) – The stream mode(s) to use.
- **`stream_subgraphs`** (`bool`, default: `False` ) – Whether to stream output from subgraphs.
- **`metadata`** (`Optional[dict]`, default: `None` ) – Metadata to assign to the run.
- **`config`** (`Optional[Config]`, default: `None` ) – The configuration for the assistant.
- **`checkpoint`** (`Optional[Checkpoint]`, default: `None` ) – The checkpoint to resume from.
- **`interrupt_before`** (`Optional[Union[All, Sequence[str]]]`, default: `None` ) – Nodes to interrupt immediately before they get executed.
- **`interrupt_after`** (`Optional[Union[All, Sequence[str]]]`, default: `None` ) – Nodes to Nodes to interrupt immediately after they get executed.
- **`feedback_keys`** (`Optional[Sequence[str]]`, default: `None` ) – Feedback keys to assign to run.
- **`on_disconnect`** (`Optional[DisconnectMode]`, default: `None` ) – The disconnect mode to use. Must be one of 'cancel' or 'continue'.
- **`on_completion`** (`Optional[OnCompletionBehavior]`, default: `None` ) – Whether to delete or keep the thread created for a stateless run. Must be one of 'delete' or 'keep'.
- **`webhook`** (`Optional[str]`, default: `None` ) – Webhook to call after LangGraph API call is done.
- **`multitask_strategy`** (`Optional[MultitaskStrategy]`, default: `None` ) – Multitask strategy to use. Must be one of 'reject', 'interrupt', 'rollback', or 'enqueue'.
- **`if_not_exists`** (`Optional[IfNotExists]`, default: `None` ) – How to handle missing thread. Defaults to 'reject'. Must be either 'reject' (raise error if missing), or 'create' (create new thread).
- **`after_seconds`** (`Optional[int]`, default: `None` ) – The number of seconds to wait before starting the run. Use to schedule future runs.

**Returns:**

- **`AsyncIterator[StreamPart]`** – AsyncIterator[StreamPart]: Asynchronous iterator of stream results.

Example Usage:
```
async for chunk in client.runs.stream(
    thread_id=None,
    assistant_id="agent",
    input={"messages": [{"role": "user", "content": "how are you?"}]},
    stream_mode=["values","debug"],
    metadata={"name":"my_run"},
    config={"configurable": {"model_name": "anthropic"}},
    interrupt_before=["node_to_stop_before_1","node_to_stop_before_2"],
    interrupt_after=["node_to_stop_after_1","node_to_stop_after_2"],
    feedback_keys=["my_feedback_key_1","my_feedback_key_2"],
    webhook="https://my.fake.webhook.com",
    multitask_strategy="interrupt"
):
    print(chunk)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

StreamPart(event='metadata', data={'run_id': '1ef4a9b8-d7da-679a-a45a-872054341df2'})
StreamPart(event='values', data={'messages': [{'content': 'how are you?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'fe0a5778-cfe9-42ee-b807-0adaa1873c10', 'example': False}]})
StreamPart(event='values', data={'messages': [{'content': 'how are you?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'fe0a5778-cfe9-42ee-b807-0adaa1873c10', 'example': False}, {'content': "I'm doing well, thanks for asking! I'm an AI assistant created by Anthropic to be helpful, honest, and harmless.", 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': 'run-159b782c-b679-4830-83c6-cef87798fe8b', 'example': False, 'tool_calls': [], 'invalid_tool_calls': [], 'usage_metadata': None}]})
StreamPart(event='end', data=None)
```

In [30]:
for chunk in client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id=assistant["assistant_id"],
    input={"messages": [{"role": "user", "content": "How are you?"}]},
    config={"configurable": {"user_id": "user001"}},
):
    pprint(chunk)

StreamPart(event='metadata', data={'run_id': '1f00f0ab-9cbd-61ec-88e7-5973181504d7', 'attempt': 1})
StreamPart(event='values', data={'messages': [{'content': 'How are you?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'bcc7bc57-9611-4b5f-882d-226e6af1eff6', 'example': False}, {'content': 'How are you?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8dd347f3-b49a-4c69-958b-ea4c5eb28647', 'example': False}], 'queries': ['How are you?']})
StreamPart(event='values', data={'messages': [{'content': 'How are you?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'bcc7bc57-9611-4b5f-882d-226e6af1eff6', 'example': False}, {'content': 'How are you?', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '8dd347f3-b49a-4c69-958b-ea4c5eb28647', 'example': False}], 'queries': ['How are you?', 'how are you?']})
StreamPart(event='values', data={'